In [4]:
import asyncio
from datetime import datetime
from dataclasses import dataclass
import csv
import timeit

@dataclass(frozen=True)
class Gpu:
    server: str
    uuid: str
    id: int
    name: str
    utilization: float
    memory_usage: int
    timestamp: datetime

    def __str__(self):
        return f"{self.server},{self.uuid},{self.id},{self.name},{self.utilization},{self.memory_usage},{self.timestamp}"

async def get_gpu_stats(server):
    proc = await asyncio.create_subprocess_exec(
        'ssh', server, 
        'nvidia-smi', 
        '--query-gpu=uuid,gpu_name,utilization.gpu,memory.used,memory.total', 
        '--format=csv,noheader,nounits',
        stdout=asyncio.subprocess.PIPE
    )
    stdout, _ = await proc.communicate()
    return stdout.decode('utf-8').splitlines()

async def gather_gpu_stats():
    servers = ['gpu3', 'gpu4', 'gpu5']
    tasks = [asyncio.create_task(get_gpu_stats(server)) for server in servers]
    results = await asyncio.gather(*tasks)
    return results

def get_gpus() -> list[Gpu]:
    gpus = []
    results = asyncio.run(gather_gpu_stats())
    for server, result in zip(['gpu3', 'gpu4', 'gpu5'], results):
        for i, stat in enumerate(csv.reader(result, delimiter=',')):
            gpus.append(
                Gpu(
                    server=server, 
                    uuid=stat[0], 
                    id=f"{i}",
                    name=stat[1][1:],
                    utilization=float(stat[2].strip('%')), 
                    memory_usage=int(int(stat[3])/int(stat[4])*100),
                    timestamp=datetime.now()
                )
            )
    return gpus

print(timeit.timeit("get_gpus()", setup="from __main__ import get_gpus", number=1))


RuntimeError: asyncio.run() cannot be called from a running event loop